In [1]:
from geopy.geocoders import Nominatim
from shapely.geometry import Point, LineString
import geopandas as gpd
import folium 
import matplotlib.pyplot as plt
import random
import pandas as pd

In [2]:
df = pd.read_csv('/Users/pauliuskomskis/Desktop/github/greenzone/data/2019/wyoming-clean.csv')
pd.set_option('display.max_rows', 1000)

In [3]:
df.info

<bound method DataFrame.info of     Unnamed: 0          city  population  violent_crime  murder  rape  \
0            0         Afton        2017              0       0     0   
1            1       Buffalo        4571              8       0     1   
2            2        Casper       57752            169       2    58   
3            3      Cheyenne       64501            220       5    43   
4            4         Cody2        9865             17       0     7   
5            5  Diamondville         756              1       0     0   
6            6       Douglas        6294              8       1     0   
7            7     Evanston2       11624             29       0    21   
8            8    Evansville        2977              7       0     0   
9            9      Gillette       31960             43       1     9   
10          10      Glenrock        2544              3       0     0   
11          11   Green River       11927             22       0     0   
12          12     

In [4]:
df.head()

,Unnamed: 0,city,population,violent_crime,murder,rape,robbery,assault,property_crime,burglary,larceny,gta,arson
0,0,Afton,2017,0,0,0,0,0,12,1,11,0,0
1,1,Buffalo,4571,8,0,1,0,7,52,13,30,9,0
2,2,Casper,57752,169,2,58,18,91,1677,285,1271,121,2
3,3,Cheyenne,64501,220,5,43,27,145,1999,211,1613,175,11
4,4,Cody2,9865,17,0,7,1,9,107,10,94,3,2


In [5]:
import re

In [6]:


df['city'] = df['city'].str.replace('\d+', '').str.strip()
df['city'] = df['city'].str.replace(",", "")
df['city'] = df['city'].str.replace("-", "")
df['city'] = df['city'].str.replace("/", "")
city_list = df['city'].astype('str')

# text = 'Township'
# head, sep, tail = text.partition('Township')


city_list.shape
pd.get_option("display.max_rows", None)
df

<ipython-input-6-19d2e1bbc19e>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['city'] = df['city'].str.replace('\d+', '').str.strip()


,Unnamed: 0,city,population,violent_crime,murder,rape,robbery,assault,property_crime,burglary,larceny,gta,arson
0,0,Afton,2017,0,0,0,0,0,12,1,11,0,0
1,1,Buffalo,4571,8,0,1,0,7,52,13,30,9,0
2,2,Casper,57752,169,2,58,18,91,1677,285,1271,121,2
3,3,Cheyenne,64501,220,5,43,27,145,1999,211,1613,175,11
4,4,Cody,9865,17,0,7,1,9,107,10,94,3,2
5,5,Diamondville,756,1,0,0,0,1,1,0,1,0,0
6,6,Douglas,6294,8,1,0,1,6,87,8,72,7,0
7,7,Evanston,11624,29,0,21,1,7,183,22,152,9,1
8,8,Evansville,2977,7,0,0,3,4,73,13,52,8,0
9,9,Gillette,31960,43,1,9,0,33,626,62,529,35,2


In [7]:
def get_coordinates(city_list):
    """Takes a list of cities and returns a dictionary of the cities and their corresponding coordinates."""
    geolocator = Nominatim(user_agent='location script')
    dicto = {}
    
    for city in city_list:
        try:
            location = geolocator.geocode(city)
        except:
            raise Exception("There was a problem with the getCoordinates function")
        coordinate_values = (location.longitude, location.latitude)  #in geopandas, the x value corresponds to the longitude while the y value, the latitude(Just in case you were wondering why it was *location.longitude, location.latitude* and not the other way round )
        dicto[city] = coordinate_values #adding the coordinate pair to the dictionary at the end of every loop
    return dicto #finally retruns the dict 

In [8]:
#getting coordinates for each city in the list
city_coords_dict = get_coordinates(city_list)
city_coords_dict

{'Afton': (-92.7835373, 44.9027452),
 'Buffalo': (-78.8783922, 42.8867166),
 'Casper': (-106.325138, 42.8501191),
 'Cheyenne': (-104.820246, 41.139981),
 'Cody': (-109.056392, 44.5263107),
 'Diamondville': (-110.538502, 41.7793918),
 'Douglas': (-88.2170516, 39.7628415),
 'Evanston': (-87.6930459, 42.0447388),
 'Evansville': (-87.5558483, 37.9747645),
 'Gillette': (-105.501876, 44.290635),
 'Glenrock': (-105.8720412, 42.8614825),
 'Green River': (-109.466473, 41.5290933),
 'Greybull': (-108.0554209, 44.488924),
 'Hanna': (-111.927479, 51.6445566),
 'Kemmerer': (-110.537711, 41.7924606),
 'Lander': (-116.9843374, 39.9059878),
 'Laramie': (-105.591101, 41.311367),
 'Lusk': (-104.452175, 42.762467),
 'Mills': (-98.5527272, 31.4861082),
 'Moorcroft': (-104.950252, 44.263315),
 'Newcastle': (-1.6131572, 54.9738474),
 'Pine Bluffs': (-104.066339, 41.18032),
 'Powell': (-113.0294193, 46.9109569),
 'Rawlins': (-101.0994722, 39.7904797),
 'Riverton': (-95.568169, 40.6870011),
 'Rock Springs': (

In [9]:
len(city_coords_dict)

32

In [10]:
df.head()

,Unnamed: 0,city,population,violent_crime,murder,rape,robbery,assault,property_crime,burglary,larceny,gta,arson
0,0,Afton,2017,0,0,0,0,0,12,1,11,0,0
1,1,Buffalo,4571,8,0,1,0,7,52,13,30,9,0
2,2,Casper,57752,169,2,58,18,91,1677,285,1271,121,2
3,3,Cheyenne,64501,220,5,43,27,145,1999,211,1613,175,11
4,4,Cody,9865,17,0,7,1,9,107,10,94,3,2


In [11]:
# crime rate per 100,000 is made as follows:
#(crime volume / population) * 100,000


#### 1. Add a total crime column

In [12]:
df['total_crime'] = df.sum(axis=1)

In [13]:
df.head()

,Unnamed: 0,city,population,violent_crime,murder,rape,robbery,assault,property_crime,burglary,larceny,gta,arson,total_crime
0,0,Afton,2017,0,0,0,0,0,12,1,11,0,0,2041
1,1,Buffalo,4571,8,0,1,0,7,52,13,30,9,0,4692
2,2,Casper,57752,169,2,58,18,91,1677,285,1271,121,2,61448
3,3,Cheyenne,64501,220,5,43,27,145,1999,211,1613,175,11,68953
4,4,Cody,9865,17,0,7,1,9,107,10,94,3,2,10119


#### 2. Making a new data frame that has only the neccesary columns for further operations

In [14]:
new_df = df.filter(['city','total_crime','population'], axis=1)
new_df

,city,total_crime,population
0,Afton,2041,2017
1,Buffalo,4692,4571
2,Casper,61448,57752
3,Cheyenne,68953,64501
4,Cody,10119,9865
5,Diamondville,765,756
6,Douglas,6490,6294
7,Evanston,12056,11624
8,Evansville,3145,2977
9,Gillette,33309,31960


### 3.  Add a column of the crime rate severity via (total crime / population) 

In [15]:
new_df['crime_index'] = (new_df['total_crime'] / new_df['population'])

In [16]:
new_df.round(5)

,city,total_crime,population,crime_index
0,Afton,2041,2017,1.01190
1,Buffalo,4692,4571,1.02647
2,Casper,61448,57752,1.06400
3,Cheyenne,68953,64501,1.06902
4,Cody,10119,9865,1.02575
5,Diamondville,765,756,1.01190
6,Douglas,6490,6294,1.03114
7,Evanston,12056,11624,1.03716
8,Evansville,3145,2977,1.05643
9,Gillette,33309,31960,1.04221


#### Next step is to create a final dataframe with following columns:
#### LATITUDE, LONGTITUDE, CRIME_INDEX

In [17]:
#lets make the coordinates dictionary into dataframe and orient it for
#latitudes and longitudes
coords_df = pd.DataFrame.from_dict(city_coords_dict, orient='index')
coords_df

,0,1
Afton,-92.783537,44.902745
Buffalo,-78.878392,42.886717
Casper,-106.325138,42.850119
Cheyenne,-104.820246,41.139981
Cody,-109.056392,44.526311
Diamondville,-110.538502,41.779392
Douglas,-88.217052,39.762842
Evanston,-87.693046,42.044739
Evansville,-87.555848,37.974764
Gillette,-105.501876,44.290635


In [18]:
coords_df.rename(columns={0: 'latitude', 1: 'longitude'}, inplace=True)

In [19]:
coords_df

,latitude,longitude
Afton,-92.783537,44.902745
Buffalo,-78.878392,42.886717
Casper,-106.325138,42.850119
Cheyenne,-104.820246,41.139981
Cody,-109.056392,44.526311
Diamondville,-110.538502,41.779392
Douglas,-88.217052,39.762842
Evanston,-87.693046,42.044739
Evansville,-87.555848,37.974764
Gillette,-105.501876,44.290635


### join original dataframe row of 'crime_index' to new df of 'coordinates'

In [20]:
coords_df['crime_index'] = new_df['crime_index'].values

In [21]:
coords_df

,latitude,longitude,crime_index
Afton,-92.783537,44.902745,1.011899
Buffalo,-78.878392,42.886717,1.026471
Casper,-106.325138,42.850119,1.063998
Cheyenne,-104.820246,41.139981,1.069022
Cody,-109.056392,44.526311,1.025748
Diamondville,-110.538502,41.779392,1.011905
Douglas,-88.217052,39.762842,1.031141
Evanston,-87.693046,42.044739,1.037164
Evansville,-87.555848,37.974764,1.056433
Gillette,-105.501876,44.290635,1.042209


In [22]:
coords_df.insert(0, 'state', 'wy')
coords_df

,state,latitude,longitude,crime_index
Afton,wy,-92.783537,44.902745,1.011899
Buffalo,wy,-78.878392,42.886717,1.026471
Casper,wy,-106.325138,42.850119,1.063998
Cheyenne,wy,-104.820246,41.139981,1.069022
Cody,wy,-109.056392,44.526311,1.025748
Diamondville,wy,-110.538502,41.779392,1.011905
Douglas,wy,-88.217052,39.762842,1.031141
Evanston,wy,-87.693046,42.044739,1.037164
Evansville,wy,-87.555848,37.974764,1.056433
Gillette,wy,-105.501876,44.290635,1.042209


In [23]:
path = '/Users/pauliuskomskis/Desktop/github/greenzone/state-by-state-coords/'
coords_df.to_csv(path+'wyoming-final.csv')